# Loop 27 Analysis: Asymmetric vs Symmetric Solutions

## Goal
Identify which N values could benefit from asymmetric layouts by analyzing:
1. Current baseline structure (is it symmetric?)
2. Per-N efficiency gaps
3. Potential for asymmetric improvement

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import matplotlib.pyplot as plt

TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def parse_s_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

def create_tree_polygon(x, y, angle):
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def get_bounding_box_side(trees):
    all_x, all_y = [], []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

# Load baseline
df = pd.read_csv('/home/submission/submission.csv')
df['x'] = df['x'].apply(parse_s_value)
df['y'] = df['y'].apply(parse_s_value)
df['deg'] = df['deg'].apply(parse_s_value)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

print(f"Loaded {len(df)} rows")
print(f"N range: {df['n'].min()} to {df['n'].max()}")
print(f"\nFirst few rows:")
print(df.head(10))

In [ ]:
# Calculate per-N scores and analyze symmetry
results = []

for n in range(1, 201):
    group = df[df['n'] == n]
    trees = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    score = (side ** 2) / n
    
    # Analyze angles - are they symmetric?
    angles = group['deg'].values % 360  # Normalize to 0-360
    unique_angles = len(np.unique(np.round(angles, 1)))
    
    # Analyze positions - check for symmetry patterns
    xs = group['x'].values
    ys = group['y'].values
    
    # Check if positions are roughly symmetric around center
    cx, cy = np.mean(xs), np.mean(ys)
    xs_centered = xs - cx
    ys_centered = ys - cy
    
    # Check for rotational symmetry (180 degree)
    rot_sym_score = 0
    for i in range(len(xs_centered)):
        # Find if there's a point at (-x, -y)
        for j in range(len(xs_centered)):
            if i != j:
                if abs(xs_centered[i] + xs_centered[j]) < 0.1 and abs(ys_centered[i] + ys_centered[j]) < 0.1:
                    rot_sym_score += 1
                    break
    rot_sym_ratio = rot_sym_score / n if n > 1 else 1.0
    
    results.append({
        'n': n,
        'side': side,
        'score': score,
        'unique_angles': unique_angles,
        'rot_sym_ratio': rot_sym_ratio
    })

results_df = pd.DataFrame(results)
print("Per-N analysis:")
print(results_df.head(20))

In [ ]:
# Calculate theoretical lower bounds and efficiency
# For a single tree, the minimum bounding box is achieved at 45 degrees
# Tree area is approximately 0.2625 (calculated from polygon)

tree_area = Polygon(TREE_TEMPLATE).area
print(f"Single tree area: {tree_area:.6f}")

# Theoretical minimum: if we could pack trees with 100% efficiency
# score = (sqrt(n * tree_area))^2 / n = tree_area
# But trees have irregular shape, so efficiency is always < 100%

results_df['theoretical_min'] = tree_area  # If 100% packing efficiency
results_df['efficiency'] = tree_area / results_df['score'] * 100

print("\nEfficiency analysis:")
print(results_df[['n', 'score', 'efficiency', 'rot_sym_ratio']].describe())

In [ ]:
# Identify N values with lowest efficiency (most room for improvement)
print("\nN values with LOWEST efficiency (most room for improvement):")
lowest_eff = results_df.nsmallest(20, 'efficiency')
print(lowest_eff[['n', 'score', 'efficiency', 'unique_angles', 'rot_sym_ratio']])

In [ ]:
# Identify N values with highest efficiency (likely already optimal)
print("\nN values with HIGHEST efficiency (likely optimal):")
highest_eff = results_df.nlargest(20, 'efficiency')
print(highest_eff[['n', 'score', 'efficiency', 'unique_angles', 'rot_sym_ratio']])

In [ ]:
# Plot efficiency by N
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.scatter(results_df['n'], results_df['efficiency'], alpha=0.5, s=10)
plt.xlabel('N')
plt.ylabel('Packing Efficiency (%)')
plt.title('Packing Efficiency by N')
plt.axhline(y=results_df['efficiency'].mean(), color='r', linestyle='--', label=f'Mean: {results_df["efficiency"].mean():.1f}%')
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(results_df['n'], results_df['score'], alpha=0.5, s=10)
plt.xlabel('N')
plt.ylabel('Score (lower is better)')
plt.title('Score by N')
plt.tight_layout()
plt.savefig('/home/code/exploration/loop27_efficiency.png', dpi=100)
plt.show()

In [ ]:
# Analyze which N values contribute most to total score
results_df['contribution'] = results_df['score']
results_df['cumulative_contribution'] = results_df['contribution'].cumsum()
results_df['pct_contribution'] = results_df['contribution'] / results_df['contribution'].sum() * 100

print("\nScore contribution by N range:")
for start, end in [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]:
    subset = results_df[(results_df['n'] >= start) & (results_df['n'] <= end)]
    total = subset['score'].sum()
    pct = total / results_df['score'].sum() * 100
    avg_eff = subset['efficiency'].mean()
    print(f"N={start:3d}-{end:3d}: Score={total:.4f} ({pct:.1f}%), Avg Efficiency={avg_eff:.1f}%")

print(f"\nTotal score: {results_df['score'].sum():.6f}")
print(f"Target: 68.919154")
print(f"Gap: {results_df['score'].sum() - 68.919154:.6f}")

In [ ]:
# Key insight: Which N values have the most potential for improvement?
# Look at N values where:
# 1. Efficiency is low (room for improvement)
# 2. Score contribution is high (impact on total)

results_df['improvement_potential'] = results_df['contribution'] * (100 - results_df['efficiency']) / 100

print("\nN values with HIGHEST improvement potential (low efficiency + high contribution):")
top_potential = results_df.nlargest(20, 'improvement_potential')
print(top_potential[['n', 'score', 'efficiency', 'improvement_potential']])

In [ ]:
# Visualize the baseline solution for a few key N values
import matplotlib.patches as patches
from matplotlib.patches import Polygon as MplPolygon

def visualize_solution(n, ax):
    group = df[df['n'] == n]
    trees = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    
    # Get bounding box
    all_x, all_y = [], []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    
    min_x, max_x = min(all_x), max(all_x)
    min_y, max_y = min(all_y), max(all_y)
    side = max(max_x - min_x, max_y - min_y)
    
    # Draw trees
    for i, tree in enumerate(trees):
        coords = list(tree.exterior.coords)
        poly = MplPolygon(coords, fill=True, alpha=0.5, edgecolor='black', linewidth=0.5)
        ax.add_patch(poly)
    
    # Draw bounding box
    cx = (min_x + max_x) / 2
    cy = (min_y + max_y) / 2
    rect = patches.Rectangle((cx - side/2, cy - side/2), side, side, 
                               fill=False, edgecolor='red', linewidth=2)
    ax.add_patch(rect)
    
    ax.set_xlim(cx - side/2 - 0.5, cx + side/2 + 0.5)
    ax.set_ylim(cy - side/2 - 0.5, cy + side/2 + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'N={n}, Score={results_df[results_df["n"]==n]["score"].values[0]:.4f}')

# Visualize some key N values
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
n_values = [1, 2, 5, 10, 20, 50, 100, 200]
for ax, n in zip(axes.flat, n_values):
    visualize_solution(n, ax)
plt.tight_layout()
plt.savefig('/home/code/exploration/loop27_solutions.png', dpi=100)
plt.show()

In [ ]:
# Summary and recommendations
print("="*70)
print("LOOP 27 ANALYSIS SUMMARY")
print("="*70)

print(f"\nCurrent total score: {results_df['score'].sum():.6f}")
print(f"Target: 68.919154")
print(f"Gap to close: {results_df['score'].sum() - 68.919154:.6f} ({(results_df['score'].sum() - 68.919154) / 68.919154 * 100:.2f}%)")

print("\n" + "="*70)
print("KEY FINDINGS:")
print("="*70)

# Find N values with lowest efficiency
low_eff = results_df[results_df['efficiency'] < 60]
print(f"\n1. N values with efficiency < 60%: {len(low_eff)}")
if len(low_eff) > 0:
    print(f"   These are: {sorted(low_eff['n'].tolist())}")

# Find N values with high contribution but low efficiency
high_impact = results_df[(results_df['efficiency'] < 70) & (results_df['score'] > 0.35)]
print(f"\n2. High-impact N values (score > 0.35, efficiency < 70%): {len(high_impact)}")
if len(high_impact) > 0:
    print(f"   These are: {sorted(high_impact['n'].tolist())}")

print("\n" + "="*70)
print("RECOMMENDATIONS:")
print("="*70)
print("\n1. Focus on N values with LOWEST efficiency - these have most room for improvement")
print("2. Try ASYMMETRIC layouts for these N values - break the symmetry")
print("3. Use GREEDY EDGE FILLING after initial placement")
print("4. Test on small N first (N=1-10) before scaling")